In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import rx_data
from numpy.linalg import eig
from scipy import linalg
# from scipy import eigh
from ReactorSlice import ReactorSlice

####################################################################################################
####### Reactor Class ##############################################################################
####################################################################################################

class Reactor:
    def __init__(self, reactor_size, slices, enrichment, rxdata, nu, simulation_time, dt, power, comps): # nu ?
        self.reactor_size = reactor_size
        self.enrich = enrichment
        self.initial_comps = comps
        self.num_slices = slices
        self.dx = float(reactor_size/(slices-1))  # - 1 because i starts from 0 ?
        self.time = 0
        self.rxdata = rxdata
        self.microxs = rxdata[0]
        self.f_yields = rxdata[1]
        self.isotopes = list(self.microxs.keys())
        self.nu = nu
        self.rx_slices = [] # [0] * self.num_slices
        self.flux = 0 ### CONSTANT at t = 0
        self.simulation_time = simulation_time
        self.dt = dt
        self.power = power
        self.build_reactor_slices()
        self.scaling_constant = 0.
        self.k = []
              
            
            
    def build_reactor_slices(self):
        
        self.rx_slices = []
        for i in range(self.num_slices):
            rx_slice = ReactorSlice(self.dx, initial_comps, self.rxdata, self.isotopes)
            self.rx_slices.append(rx_slice)
            print('rx_slice: ', rx_slice)
        
    
    
    def build_constant_array(self, time):

        constants_array = []        
        D = [0] * len(self.rx_slices)
        Sigma_a = [0] * len(self.rx_slices)
        Sigma_f = [0] * len(self.rx_slices)
        for sli in self.rx_slices: 
            isotope_constants = sli.calculate_constants(time)
            constants_array.append(isotope_constants)
        D = constants_array[0]
        Sigma_a = constants_array[1]
        Sigma_f = constants_array[2]
        return D, Sigma_a, Sigma_f
        
    
    
    def build_diffusion_matrix(self, D, Sigma_a):
        
        diff_matrix = []
        dx = self.dx
        for i in (1, ((self.num_slices)+1)): 
            row = [0] * len(D)
            if i == 1:
                row[i-1] = - ((D[i-1] + 2*D[i] + D[i+1]) / (2*(dx**2))) - Sigma_a[i-1]
                row[i] = ((D[i] + D[i+1]) / (2*(dx**2)))
            elif i == len(D):
                row[i-2] = (D[i-2] + D[i-1]) / (2*(dx**2))
                row[i-1] = - ((D[i-2] + 2*D[i-1] + D[i]) / (2*(dx**2))) - Sigma_a[i-1]
            else:
                row[i-2] = ((D[i-1] + D[i]) / (2*(dx**2)))
                row[i-1] = -((D[i-1] + 2*D[i] + D[i+1]) / (2*(dx**2))) - Sigma_a[i]
                row[i] = ((D[i] + D[i+1]) / (2*(dx**2)))  
            diff_matrix.append(row)
            return
        return diff_matrix
    
    #############################################################
       # diff_matrix = []
       # dx = self.dx
       # for i in (1, (self.num_slices+1)): # Columns
       #     row = [0] * self.num_slices
       #     if i == 1:
       #         row[i-1] = - ((D[i-1] + 2*D[i] + D[i+1]) / (2*(dx**2))) - Sigma_a[i-1]
       #         row[i] = ((D[i] + D[i+1]) / (2*(dx**2)))
       #     elif i == num_slices:
       #         row[i-2] = (D[i-2] + D[i-1]) / (2*(dx**2))
       #         row[i-1] = - ((D[i-2] + 2*D[i-1] + D[i]) / (2*(dx**2))) - Sigma_a[i-1]
       #     else:
       #         row[i-2] = ((D[i-1] + D[i]) / (2*(dx**2)))
       #         row[i-1] = -((D[i-1] + 2*D[i] + D[i+1]) / (2*(dx**2))) - Sigma_a[i]
       #         row[i] = ((D[i] + D[i+1]) / (2*(dx**2)))  
       #     diff_matrix.append(row)
       #     return
       # return diff_matrix
    #############################################################
    
    
    def build_fission_matrix(self, Sigma_f): # need nu as well ?
        
        fiss_matrix = []
        for i in range(self.num_slices):
            roww = [0] * (self.num_slices)
            roww[i-1] = self.nu * Sigma_f[i-1]   ### nu[i-1] ????? why did that work??? did that work???
            fiss_matrix.append(roww)

        return fiss_matrix
        
    
    
    def calculate_scaled_flux(self, flux):
        #scales the flux to match power demand
        # M * phi = (1/lambda) * phi
        return
        
    
    
    def calculate_flux(self, time):  # FLUX PROFILE* across reactor at single timestamp ???
        #performs the required operations to solve the diffusion matrix and scale the flux
        
        D, Sigma_a, Sigma_f = self.build_constant_array(time)   
        diffusion_matrix = self.build_diffusion_matrix(D, Sigma_a)
        fission_matrix = self.build_fission_matrix(Sigma_f)
        
       # M = diffusion_matrix * fission_matrix**-1
    
        ########################### PROBLEM HERE #######################
        eigenvalue, eigenvector = np.linalg.eig(diffusion_matrix, fission_matrix) #########
        ########################### PROBLEM HERE #######################
        ################################################################
        
        ###
        return eigenvalue, eigenvector   ## test --> returns to control kernel
        ###
        
        ################################################################
        ################################################################
        
        scaler = 1 # <--- proper eigen value goes here
        
        lammda = 1 / scaler
        
        Power_total = 1 # W / cm^{3} = MW / m^{3}
        
        Energy_released = 3.20e-11 # J   ---> = ~200 MeV
        
        flux_array = []
        
        for i in range(len(self.rx_slices)):
            slice_flux = ( (( self.power / self.num_slices ) * scaler) / ( Sigma_f * Energy_released ) )
            flux_array.append(slice_flux)
            print('Flux for slice (', i, '): ', slice_flux)
            
        return flux_array
    
       
              
    def calculate_criticality(self):
        
      #calculates K # n produced / n absorbed
        for sli in (self.rxslices):
            #
            #
            return
        
        return
    
        

    def burn_reactor(self, time):
        #pushes the reactor forward in time
        
       # D, Sigma_a, Sigma_f = self.rx_slices[i].calculate_constants(time)
        # [self.D, self.Sigma_a, self.Sigma_f] = build_constant_array(time)
       # flux = self.rx_slices.calculate_flux(self.time)
        #solve matric
       # for t in range(self.simulation_time): 
            
       #     for i in self.rx_slices: # in each timestep   # range(self.rxslices) ???
           #     props = ReactorSlice.calculate_constants(self, time)
                
            
        # while loop
            # calc properties
            # calc flux  
            # matrix solve
            return

############################################################################################################
############################################################################################################

In [2]:
####################################################################################################
####### REACTOR SLICE ##############################################################################
####################################################################################################

class ReactorSlice:
    def __init__(self, dx, compositions, rxdata, nuclides):
        self.evolution_composition = {}
        self.evolution_composition[0] = compositions
        self.dx = dx
        self.microxs = rxdata[0] # fission microscopic cross section
        self.yields = rxdata[1] # absorbtion microscopic cross section
        self.rrm = self.compute_reaction_rate_matrix # (self.microxs, self.yields)
        self.nuclides = nuclides
        self.D = None
        self.Sigma_a = None
        self.Sigma_f = None
        self.flux = 0.
        print('New Reactor Slice Created!')
       

    
    def calculate_D(self, time):
        Sigma_s_total = 0.
        for i in range(len(self.evolution_composition[time])):
            Sigma_s_total += self.evolution_composition[time][i]*self.microxs[self.nuclides[i]]['xs_scattering']*1.E-24
        D_total = 1/(3*Sigma_s_total)
        self.D = D_total

        
    
    def calculate_Sigma_a(self, time):
        Sigma_a_total = 0.
        for i in range(len(self.evolution_composition[time])):
            Sigma_a_total += self.evolution_composition[time][i]*self.microxs[self.nuclides[i]]['xs_absorption']*1.E-24
        self.Sigma_a = Sigma_a_total

        
       
    def calculate_Sigma_f(self, time):
        Sigma_f_total = 0.
        for i in range(len(self.evolution_composition[time])):
            Sigma_f_total += self.evolution_composition[time][i]*self.microxs[self.nuclides[i]]['xs_fission']*1.E-24
        self.Sigma_f = Sigma_f_total

        
        
    def calculate_constants(self, time):
        self.calculate_D(time)
        self.calculate_Sigma_a(time)
        self.calculate_Sigma_f(time)
        return [self.D, self.Sigma_a, self.Sigma_f]

    
    
    def compute_reaction_rate_matrix(self, xs, fy): # B ???????
        
        # self.flux = #####
        
        rrm = np.zeros((6, 6))
        # U235
        rrm[0, 0] = -self.flux * ( xs['U235']['xs_absorption'] + xs['U235']['xs_fission'] ) * 1.E-24
        
        # U238
        rrm[1, 1] = -self.flux * xs['U238']['xs_absorption'] * 1.E-24
        
        # Pu239
        rrm[2, 2] = -self.flux * (xs['Pu239']['xs_absorption'] + xs['Pu239']['xs_fission']) * 1.E-24
        rrm[2, 1] = self.flux * xs['U238']['xs_absorption'] * 1.E-24
        
        # Xe135
        rrm[3, 3] = -self.flux * xs['Xe135']['xs_absorption'] * 1.E-24 - xs['Xe135']['lambda']
        rrm[3, 0] = self.flux * fy['Xe135'] * xs['U235']['xs_fission'] * 1.E-24
        rrm[3, 2] = self.flux * fy['Xe135'] * xs['Pu239']['xs_fission'] * 1.E-24
        
        # Lumped fission products
        rrm[4, 0] = self.flux * (2 - fy['Xe135']) * xs['U235']['xs_fission'] * 1.E-24
        rrm[4, 2] = self.flux * (2 - fy['Xe135']) * xs['Pu239']['xs_fission'] * 1.E-24
        rrm[4, 3] = self.flux * xs['Xe135']['xs_absorption'] * 1.E-24 + xs['Xe135']['lambda']
        
        self.rrm = rrm

        
        
    def evolution_solver(self, dt, time):
        rrm_id = np.identity(len(self.evolution_composition[time])) - (dt * self.rrm)
        # Ax = B --> A : rrm_id, B : previous step composition
        compositions = np.linalg.solve(rrm_id, self.evolution_composition[time])
        self.evolution_composition[time+dt] = compositions
        
############################################################################################################
############################################################################################################

In [3]:
####################################################################################################
####### Control ####################################################################################
####################################################################################################

# one function to rule them all
# one function to crawl them
# one function to bring them all
# and in this kernel, call them
                            
def run(rx_data, comps):
    reactor_size = 1       # m
   # reactor_size = 2       # m  # HW IV 
    n_slices = 4           # 100 # number of slices
    enrichment = 0.03       # x
    nu = 2.6               # CONFIRM real value.... average across fissile isotopes # use?
    data = [rx_data.microscopic_cross_sections, rx_data.fission_yield]  # why are these the same
    run_time = 1440 # minutes per day
    time_step = 1 # 1 minute
    power = 1 # 1 MW/m^{3} = 1e6 W/m^{3} = 1 W/cm^{3}
    Rex = Reactor(reactor_size, n_slices, enrichment, data, nu, run_time, time_step, power, comps)
    Rex.burn_reactor(0)
    #reactor_size, slices, enrichment, rxdata, simulation_time, dt, power
    return Rex # return Rex.rx_slices (?)

N_U235 = ( ( 0.03 * 18.65 * 6.022e23 ) / (235) ) * 0.5           # initial    #/cm^{3} !!!!!!!
N_U238 = ( ( (1 - 0.03) * 18.65 * 6.022e23 ) / (238) ) * 0.5   
N_Pu239 = 1e-9
N_Xe135 = 1e-9
N_LFP = 1e-9
N_H2O = 0.5
initial_comps = [ N_U235, N_U238, N_Pu239, N_Xe135, N_LFP, N_H2O ] # self. ?
# N_Pu240 = 1e-9
# N_Pu241 = 1e-9

Rex = run(rx_data, initial_comps)

time = 0
eigen_test = Rex.calculate_flux(time)
print('eigen_test: ', eigen_test)


#constants = Rex.build_constant_array(0)
#print('constants array:', constants)


#Rex, rex_slice = run(0)
#Rex.D = Rex.rxslices.calculate_D(time)
#diffusion_matrix = Rex.build_diffusion_matrix(Rex.D, Rex.Sigma_a)
#print('diffusion matrix: ', diffusion_matrix)

####################################################################################################
####################################################################################################

New Reactor Slice Created!
rx_slice:  <__main__.ReactorSlice object at 0x7ff934c55040>
New Reactor Slice Created!
rx_slice:  <__main__.ReactorSlice object at 0x7ff934c552e0>
New Reactor Slice Created!
rx_slice:  <__main__.ReactorSlice object at 0x7ff934c3a040>
New Reactor Slice Created!
rx_slice:  <__main__.ReactorSlice object at 0x7ff934c3a640>


TypeError: _unary_dispatcher() takes 1 positional argument but 2 were given

In [ ]:
####################################################################################################
####### Test #######################################################################################
####################################################################################################



In [52]:
# Reactor Specs

X = 1
num_slices = 4 # 100 # number of slices
dx = X / num_slices
enrich = 0.03
data = [rx_data.microscopic_cross_sections, rx_data.fission_yield]
isotopes = list(rx_data.microscopic_cross_sections.keys())

N_U235 = ( ( enrich * 18.65 * 6.022e23 ) / (235) ) * 0.5           # initial    #/cm^{3} !!!!!!!
N_U238 = ( ( (1 - enrich) * 18.65 * 6.022e23 ) / (238) ) * 0.5   
N_Pu239 = 10 * 0.5
N_Xe135 = 10 * 0.5
N_LFP = 10 * 0.5
N_H2O = ((1 * 6.022e23)/(2 + 15.999)) * 0.5
#N_Pu240 = 1e-9
#N_Pu241 = 1e-9
initial_comps = [ N_U235, N_U238, N_Pu239, N_Xe135, N_LFP, N_H2O ] # N_Pu240, N_Pu241] 

#for i in range(len(initial_comps)):
#    print(initial_comps[i])
print('Isotopes : ', isotopes ) # list of keys
print('Initial Compositions:', initial_comps)
print(' ')
# print('data[0] :', data[0])
# print('rx_data.microscopic_cross_sections.keys() :', rx_data.microscopic_cross_sections.keys())
# print(rx_data.microscopic_cross_sections.keys)
# print(xs['U235']['xs_absorption'])
# print(rx_data.microscopic_cross_sections['U235']['xs_absorption'] * 1e-24)

Sig_s_U235 = N_U235 * (rx_data.microscopic_cross_sections['U235']['xs_scattering'] * 1e-24 )
Sig_s_U238 = N_U238 * (rx_data.microscopic_cross_sections['U238']['xs_scattering'] * 1e-24 )
Sig_s_Pu239 = N_Pu239 * (rx_data.microscopic_cross_sections['Pu239']['xs_scattering'] * 1e-24 )
Sig_s_Xe135 = N_Xe135 * (rx_data.microscopic_cross_sections['Xe135']['xs_scattering'] * 1e-24 )
Sig_s_LFP = N_LFP * (rx_data.microscopic_cross_sections['LFP']['xs_scattering'] * 1e-24 )
Sig_s_H2O = N_H2O * (rx_data.microscopic_cross_sections['H2O']['xs_scattering'] * 1e-24 )
# Sig_s_Pu240 = N_Pu240 * (rx_data.microscopic_cross_sections['Pu240']['xs_scattering'] * 1e-24)
# Sig_s_Pu241 = N_Pu241 * (rx_data.microscopic_cross_sections['Pu241']['xs_scattering'] * 1e-24)
Sigmas_s = [Sig_s_U235, Sig_s_U238, Sig_s_Pu239, Sig_s_Xe135, Sig_s_LFP, Sig_s_H2O]
Sigma_s_total = np.sum(Sigmas_s)
print('Sigmas_s : ', Sigmas_s)
print(' ')
print('Total Sigma_s: ', Sigma_s_total)
print(' ')

D_U235 = 1 / (3 * Sig_s_U235)
D_U238 = 1 / (3 * Sig_s_U238)
D_Pu239 = 1 / (3 * Sig_s_Pu239)
D_Xe135 = 1 / (3 * Sig_s_Xe135)
D_LFP = 1 / (3 * Sig_s_LFP)
D_H2O = 1 / (3 * Sig_s_H2O)
# D_Pu240 = 1 / (3 * Sig_s_Pu240)
# D_Pu241 = 1 / (3 * Sig_s_Pu241)
D = [D_U235, D_U238, D_Pu239, D_Xe135, D_LFP, D_H2O]
D_total = np.sum(D)
print('Ds     : ', D)
print(' ')
print('Total D: ', D_total)
print(' ')

Sig_a_U235 = N_U235 * (rx_data.microscopic_cross_sections['U235']['xs_absorption'] * 1e-24 )
Sig_a_U238 = N_U238 * (rx_data.microscopic_cross_sections['U238']['xs_absorption'] * 1e-24 )
Sig_a_Pu239 = N_Pu239 * (rx_data.microscopic_cross_sections['Pu239']['xs_absorption'] * 1e-24 )
Sig_a_Xe135 = N_Xe135 * (rx_data.microscopic_cross_sections['Xe135']['xs_absorption'] * 1e-24 )
Sig_a_LFP = N_LFP * (rx_data.microscopic_cross_sections['LFP']['xs_absorption'] * 1e-24 )
Sig_a_H2O = N_H2O * (rx_data.microscopic_cross_sections['H2O']['xs_absorption'] * 1e-24 )
# Sig_a_Pu240 = N_Pu240 * (rx_data.microscopic_cross_sections['Pu240']['xs_absorption'] * 1e-24 )
# Sig_a_Pu241 = N_Pu241 * (rx_data.microscopic_cross_sections['Pu241']['xs_absorbtion'] * 1e-24 )
Sigmas_a = [Sig_a_U235, Sig_a_U238, Sig_a_Pu239, Sig_a_Xe135, Sig_a_LFP, Sig_a_H2O]
Sigma_a_total = np.sum(Sigmas_a)
print('Sigmas_a : ', Sigmas_a)
print(' ')
print('Total Sigma_a: ', Sigma_a_total)
print(' ')

Sig_f_U235 = N_U235 * (rx_data.microscopic_cross_sections['U235']['xs_fission'] * 1e-24 )
Sig_f_U238 = N_U238 * (rx_data.microscopic_cross_sections['U238']['xs_fission'] * 1e-24 )
Sig_f_Pu239 = N_Pu239 * (rx_data.microscopic_cross_sections['Pu239']['xs_fission'] * 1e-24 )
Sig_f_Xe135 = N_Xe135 * (rx_data.microscopic_cross_sections['Xe135']['xs_fission'] * 1e-24 )
Sig_f_LFP = N_LFP * (rx_data.microscopic_cross_sections['LFP']['xs_fission'] * 1e-24 )
Sig_f_H2O = N_H2O * (rx_data.microscopic_cross_sections['H2O']['xs_fission'] * 1e-24 )
# Sig_f_Pu240 = N_Pu240 * (rx_data.microscopic_cross_sections['Pu240']['xs_fission'] * 1e-24 )
# Sig_f_Pu241 = N_Pu241 * (rx_data.microscopic_cross_sections['Pu241']['xs_fission'] * 1e-24 )
Sigmas_f = [Sig_f_U235, Sig_f_U238, Sig_f_Pu239, Sig_f_Xe135, Sig_f_LFP, Sig_f_H2O]
Sigma_f_total = np.sum(Sigmas_f)
print('Sigmas_f : ', Sigmas_f)
print(' ')
print('Total Sigma_f: ', Sigma_f_total)
print(' ')

for i in range(num_slices):
    print(' ')
    print('Slice: (', (i+1), ')')
    print(' ')

    #isotope_array = []

    #for i in (self.nucs):
    #    slice_constants = calculate_constants(time)
    #    isotope_array.append(slice_constants)

    # print('Isotope Array:', isotope_array)

    # Diffusion Matrix

    diffusion_matrix = [] # 
    for i in range(len(isotopes)+1): # Columns
        row = [0] * (len(isotopes)+1)
        if i == 0:
            row[i] = - ((0 + 2*D[i] + D[i+1]) / (2*(dx**2))) - Sigmas_a[i]
            row[i+1] = ((D[i] + D[i+1]) / (2*(dx**2)))
        elif i == len(isotopes):
            row[i-1] = (D[i-1] + D[i]) / (2*(dx**2))
            row[i] = - ((D[i-1] + 2*D[i] + 0) / (2*(dx**2))) - Sigmas_a[i]
        else:
            row[i-1] = ((D[i-1] + D[i]) / (2*(dx**2)))
            row[i] = -((D[i-1] + 2*D[i] + D[i+1]) / (2*(dx**2))) - Sigmas_a[i]
            row[i+1] = ((D[i] + D[i+1]) / (2*(dx**2)))  
        diffusion_matrix.append(row)
        print('Diffusion Matrix Row (', i, '):', row)
        print(' ')
        # eigenvalue, eigenvector = eig(diffusion_matrix) #eigen value of M, this is A
    print(' ')

    # print('Diffusion Matrix:', diffusion_matrix)

    # flux first ???
    # assume value?  constant in hint

    flux = 1e12
    #


    # Fission Matrix
    nu = 2.6 # confirm

    fission_matrix = []
    for i in range(len(isotopes)):
        roww = [0] * len(isotopes)
        roww[i] = nu * Sigmas_f[i]
       # row[i] = self.nu * Sigma_f[i] ### sigma_f[i]   # generalize nu value !!!!
        fission_matrix.append(roww)
        print('Fission Matrix Row (', (i+1), '):', roww)
        print(' ')
        
#eigenvalue
eigenvalue, eigenvector = linalg.eig(diffusion_matrix, b=fission_matrix) #eigen value of M, this is A

print('eigenvalue: ', eigenvalue)
print(' ')
print('eigenvector: ', eigenvector)
print(' ')

Isotopes :  ['U235', 'U238', 'Pu239', 'Xe135', 'LFP', 'H2O']
Initial Compositions: [7.168742553191488e+20, 2.2886762815126053e+22, 5.0, 5.0, 5.0, 1.6728707150397244e+22]
 
Sigmas_s :  [0.010839138740425528, 0.21284689418067226, 4.4064999999999996e-23, 2.1499999999999997e-23, 4.9999999999999995e-31, 1.723056836490916]
 
Total Sigma_s:  1.946742869412014
 
Ds     :  [30.7527508703378, 1.5660709291364512, 7.564582624153713e+21, 1.550387596899225e+22, 6.666666666666668e+29, 0.1934546361292306]
 
Total D:  6.666666897351254e+29
 
Sigmas_a :  [0.4831015606595743, 0.05950558331932774, 5.099999999999999e-21, 1.1999999999999998e-22, 0.0, 0.01110786154786377]
 
Total Sigma_a:  0.5537150055267659
 
Sigmas_f :  [0.41944312678723394, 0.0, 3.7369999999999995e-21, 0.0, 0.0, 0.0]
 
Total Sigma_f:  0.41944312678723394
 
 
Slice: ( 1 )
 
Diffusion Matrix Row ( 0 ): [-505.05568291915597, 258.550574395794, 0, 0, 0, 0, 0]
 
Diffusion Matrix Row ( 1 ): [258.550574395794, -6.05166609932297e+22, 6.05166609932

IndexError: list index out of range